
### Information
Basic Implementation for MNIST and CNN was modifed from [here](https://medium.com/@nutanbhogendrasharma/pytorch-convolutional-neural-network-with-mnist-dataset-4e8a4265e118). 

### Extensions
Re-implement without auto-differentation and calculate back-propagation.

In [3]:
"""Load the MNIST Dataset"""

from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Instantiate dataset objects
train_data = datasets.MNIST(
    root='data', train=True, transform=ToTensor(), download=True)
test_data = datasets.MNIST(
    root='data',train=False, transform=ToTensor())

# set dataloaders
train_loader = DataLoader(
    train_data, batch_size=100, shuffle=True, num_workers=1)
test_loader = DataLoader(
    test_data, batch_size=100, shuffle=True, num_workers=1)

In [33]:
"""
Build a convolutional network. 
"""

import torch
import torch.nn as nn 
from torch.autograd import Variable

class CNN(nn.Module):
    """Convolutional Neural Network (with custom layer)"""
    def __init__(
        self, stride=1, padding=2, kernel_size=5):
        super(CNN, self).__init__()

        # create convolutional blocks
        self.conv1 = nn.Sequential(         
            custom_Conv2d(1, 16, kernel_size, stride, padding),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            custom_Conv2d(16, 32, kernel_size, stride, padding),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )   

        # outputs: 10 classes
        self.out = nn.Linear(32*7*7, 10)

    def instantiate(self, loss_func, optimizer):
        self.loss_func = loss_func
        self.optimizer = optimizer

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = torch.flatten(x, start_dim=1)    
        x = self.out(x)
        return x


def train_cnn(num_epochs, model, train_loader, device="cpu"):
    """Training Loop for CNN"""
    
    model.train()
    for epoch in range(num_epochs):
        loss = 0 
        for i, (images, labels) in enumerate(train_loader):
            
            # make predictions
            output = model(images.to(device))             
            loss = model.loss_func(output, labels.to(device))
            
            # backpropagation   
            model.optimizer.zero_grad()            
            loss.backward()          
            model.optimizer.step()                
            
            # log progress
            loss += float(loss.item())

        print(f'Epoch: [{epoch+1}/{num_epochs}], Loss {loss.item():.4f}')

def test_cnn(model, test_loader, device="cpu"):
    """Testing loop for CNN"""

    model.eval()
    with torch.no_grad():
        correct = 0
        for images, labels in test_loader:
            test_output = model(images.to(device))
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            correct += float((pred_y == labels).sum().item())  

        print(f'Accuracy: {correct/len(test_loader.dataset)*100}')

# set parameters
device = "cpu"
epochs = 1

# instantiate the model
model = CNN()
model.to(device)
model.instantiate(
    loss_func=nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(
        model.parameters(), lr=0.01) 
)

# run training and test loop
train_cnn(epochs, model, train_loader, device=device)
test_cnn(model, test_loader, device=device)

Epoch: [1/1], Loss 0.0707
Accuracy: 97.68


In [38]:
"""Visualise Predictions"""

# get a small sample
sample = next(iter(test_loader))
imgs, lbls = sample

# compare to actual predictions
actual_number = lbls[:10].numpy()
test_output = model(imgs[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()

# visualise
print(f'Prediction number: {pred_y}')
print(f'Actual number: {actual_number}')

Prediction number: [5 8 3 6 1 3 0 1 9 7]
Actual number: [5 8 3 6 1 3 0 1 9 7]
